<a href="https://colab.research.google.com/github/fdavenport/CIVE480A6-climate-change-impacts/blob/main/lectures/08_Climate_Change_Assessment_Arkansas_River.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIVE 480A6: Climate Change Risks and Impacts
## Week 13: Climate Change Impact Assessment for Arkansas River Stream Temperature

This week's objectives:
1. Analyze the relationship between stream temperature extremes and air temperature in the Arkansas River near Canon City, CO.
2. Analyze downscaled climate projections from the NASA Earth Exchange Global Daily Downscaled Projections (NEX-GDDP-CMIP6) dataset.
3. Use the "delta" change bias correction method to calculate the future temperature changes for the Arkansas River basin upstream of Canon City, CO.  


## Part 1: Finding the relationship between stream temperature and air temperature

Stream temperature is an important indicator of water quality and of the health of aquatic ecosystems. According to the [EPA](https://www.epa.gov/climate-indicators/climate-change-indicators-stream-temperature), "higher temperatures reduce levels of dissolved oxygen in the water, which can negatively affect the growth and productivity of aquatic life. Persistently warmer temperatures in streams can accelerate natural chemical reactions and release excess nutrients into the water."

Despite the fact that stream temperature is a very important quantity to consider, we don't have readily available future projections of stream temperature. Instead, we will look at how stream temperature relates to air temperature. Then we will be able to draw some conclusions about how stream temperature will be affected by climate change in the future.


For this assignment, we will analyze stream temperature data from [USGS streamgage 07096000 on the Arkansas River near Canon City, CO](https://waterdata.usgs.gov/nwis/uv?site_no=07096000&legacy=1).

In [ ]:
import pandas as pd

In [ ]:
## load stream temperature data

water_temp_url = "https://raw.githubusercontent.com/fdavenport/CIVE480A6-climate-change-impacts/refs/heads/main/lectures/data/usgs07096000_arkansas_river_canon_city.csv"

water_temp = pd.read_csv(water_temp_url)

# convert time column to datetime format
water_temp["time"] = pd.to_datetime(water_temp["time"])

**1a) Discussion:** What time period does the water temperature data cover?

*add answer here*

**1b) Discussion:** What was the maximum water temperature measured?  

In [ ]:
# add code here

In addition to the streamflow data, we will analyze air temperature and conditions in the watershed upstream. We will use a new library called **geopandas** to load the watershed boundary file:

In [ ]:
import geopandas as gpd

In [ ]:
watershed_url = "https://raw.githubusercontent.com/fdavenport/CIVE480A6-climate-change-impacts/refs/heads/main/lectures/data/usgs_07096000_watershed.shp"

watershed = gpd.read_file(watershed_url)

Next we need to load historical temperature data from the PRISM dataset (which we have used in previous lectures and homeworks).

**1c)** Read in the prism air temperature and precipitation data:

In [ ]:
!pip install cftime
import cftime

import xarray as xr

In [ ]:
prism_temp_url = "https://raw.githubusercontent.com/fdavenport/CIVE480A6-climate-change-impacts/main/lectures/data/PRISM_CO_monthly_temp_1994_2023.nc"

!wget {prism_temp_url} -O PRISM_CO_monthly_temp_1994_2023.nc

In [ ]:
## read in data
prism_temp = xr.open_dataset("PRISM_CO_monthly_temp_1994_2023.nc")


**1d)** Make a map of the air temperature in June 2014. Add the watershed boundary (some example code is given).

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()

# add code here
# hint: the code to make a map looks like this:
# temp = ax.pcolormesh(lon_info, lat_info, variable_info, cmap = "colormap of your choosing")
# plt.colorbar(temp, label = "label text")


## add watershed boundary
watershed.boundary.plot(ax = ax, edgecolor = "black")


The air temperature data that we just loaded is for the entire state of Colorado. However, we are only interested in the data for the Arkansas River watershed upstream of the streamgage location. We need to "clip" the data so that only information inside the watershed is included.

**1e)** Clip the air temperature data so that only information inside the watershed is included (example code provided). Make a plot of the clipped data for June 2014.

In [ ]:
!pip install rioxarray
import rioxarray

In [ ]:
# clip temperature data

## add geographic location information
prism_temp = prism_temp.rename({"lon":"longitude", "lat":"latitude"})
prism_temp.rio.write_crs("EPSG:4269", inplace=True)

# clip to basin geometry
prism_temp = prism_temp.rio.clip(watershed.geometry)

In [ ]:
fig, ax = plt.subplots()

# make a map of the clipped data


# remember to label your figure!

Now that we've clipped the data for the Arkansas River Watershed, we will calculate the average summer air temperature. To simplify things, this code has already been written for you:

In [ ]:
## calculate average summer air temperature for each year

## select summer months
summer_temp = prism_temp.where(prism_temp.time.dt.month.isin([5, 6, 7, 8]))

# calculate summer average
summer_temp = summer_temp.groupby(summer_temp.time.dt.year).mean(dim = ["time", "latitude", "longitude"])


**1f)** What months were used to calculate the summer air temperature?

*add answer here*

We will also calculate the number of days each year where the maximum water temperature exceeds 72F. While there is no universal threshold for stream temperature impacts, many Colorado fish species will begin to experience additional stress above ~70F. To simplify things, this code has already been written for you.

In [ ]:
## add a water year variable to dataframe
water_temp["wateryear"] = np.where(water_temp.time.dt.month >=10, water_temp.time.dt.year+1, watertemp.time.dt.year)

## calculate number of days with max temp > 72F each water year
days_above_72F = pd.DataFrame(water_temp.groupby('wateryear')["maxT_degF"].apply(lambda x: (x > 72).sum()))


**1g)** Make a plot of the number of days with max stremp temperature above 72F vs. summer air temperature:

In [ ]:
fig, ax = plt.subplots()




## remember to label your plot!

**1h) Discussion**: describe the relationship between stream temperature and summer air temperature based off of your graph.

*add answer here*

## **STOP**. Wait here and raise your hand to let me know you are at this point.

## Part 2: Analyzing Downscaled Climate Projections

In Part 1, we saw that there is a clear relationship between stream temperature and air temperature. In this section, we will analyze future projections of air temperature and precipitation from the NASA Global Daily Downscaled Projections (NEX-GDDP-CMIP6) dataset. This dataset uses statistical downscaling to create higher resolution climate projections from the same climate modeling simulations we've already analyzed.

The downscaled data associated with the CESM2 climate model projections has been uploaded to the github site. We will analyze one future climate change scenario that has "moderate" emissions over the next 30 years (ssp 370). The data from the historical climate model simulation and the future simulation have been combined into the same file.

**2a)** Download and read in the air temperature data from the downscaled climate model simulations:

In [ ]:
nex_gddp_url = "https://raw.githubusercontent.com/fdavenport/CIVE480A6-climate-change-impacts/main/lectures/data/NEX-GDDP-CMIP6_CESM2_r4i1p1f1_historical_ssp370_tas.nc"

!wget {nex_gddp_url} -O NEX-GDDP-CMIP6_CESM2_r4i1p1f1_historical_ssp370_tas.nc

In [ ]:
## read in the data

climate_model_temp = xr.open_dataset("NEX-GDDP-CMIP6_CESM2_r4i1p1f1_historical_ssp370_tas.nc")


**2b) Discussion:** What is the spatial resolution of the downscaled climate model data?

*add answer here*

**2c**) Make a plot of the downscaled temperature data for June 2014.  

In [ ]:
fig, ax = plt.subplots()

# add code here


**2d)** Clip the climate model air temperature data to the boundary of the Arkansas River watershed (same steps as above).

In [ ]:
# add code here



**2e)** Calculate the summer air temperature for each year (use same code as above):

In [ ]:
## add code here



**2f)** Make a time series graphs of summer air temperature for the Arkansas River Watershed:

In [ ]:
fig, axes = plt.subplots()

# add code here



## **STOP**. Wait here and raise your hand to let me know you are at this point.

## Part 3: Calculating Future Temperature "Delta" for the Arkansas River

Even though we are using downscaled climate model data that is higher resolution than the original climate model projections, we still need to account for model bias (aka model error).

We will do this by using the "delta" change method. More specifically, we will calculate the change in summer air temperature for the next 30 years according to the climate models.

**3a)** Calculate the average summer air temperature from 1994-2024 and for 2025-2055.

In [ ]:
## add code here




**3b)** What is the change in average summer air temperature between the two periods?

In [ ]:
# add code here



### **STOP**. Wait here and raise your hand to let me know you are at this point.

## Part 4: Assessing Impacts on Stream Temperature

In this section, we will try to draw some conclusions about how increases in air temperature will affect stream temperature in the Arkansas River.


**4a)** Add the "delta" that you calculated in part 3 to the prism summer temperature data to calculate an estimate of future Arkansas River air temperature.

In [ ]:
## add code here



From the graphs in 1f, we see that days with stream temperatures above 72F become more likely when the summer air temperature is above ~13C. And, if the summer air temperature is above ~14C, the number of 72-degree water temperature days is even higher. This result suggests that average summer temperatures above 13C and 14C would indicate worsening stream temperature.

**4b)** In the "future prism data", calculate the number of years with average summer temperatures exceeding 13C.

In [ ]:
# add code here



**4c)** In the "future prism data", calculate the number of years with average summer temperatures exceeding 14C.

In [ ]:
# add code here



**4d) Discussion:** Based on your results, how would you say that stream temperature would be affected in the Arkansas River under the SSP3-70 future climate change scenario?

*add answer here*

While this analysis doesn't give us exact predictions of future stream temperature, it does allow us to understand the potential implications of air temperature changes on stream temperature. If there is a risk of hotter stream temperatures in the future, river restoration efforts might be able to help adapt to these changes by adding vegetation that can shade the river, adding deeper pools where water stays cooler, or by increasing the dissolved oxygen in the stream. Additionally, river recreation policies could be updated to limit fishing on hot days when fish are already stressed.

We could also do a more advanced analysis where we look at how stream temperature relates to other factors like the amount of snowmelt and precipitation during the water year. Years with less water tend to have hotter stream temperatures, because less water heats up more quickly. So, instead of analyzing only air temperature, we could look at how stream temperature would be affected by all of these variables (air temperature, snowmelt, and precipitation) in the future.